In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pystan
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)
%matplotlib inline

In [2]:
#season_cresults = pd.read_csv('../input/wdatafiles/WRegularSeasonCompactResults.csv')
season_cresults = pd.read_csv('../input/Stage2WDataFiles/WRegularSeasonCompactResults.csv')

In [3]:
season=season_cresults[season_cresults["Season"]==2019]

In [5]:
#場所情報をダミー変換
season=pd.concat([season,pd.get_dummies(season["WLoc"])],axis=1)
del season["WLoc"]

In [7]:
LW=season[["LTeamID","WTeamID"]].astype(int)
LW-=3100 #0始まりにするため

In [8]:
#参加チーム数
teamnum=len(pd.concat([season["LTeamID"],season["WTeamID"]],axis=0).unique())

In [10]:
model="""
data {
  int N;  // num of players
  int G;  // num of games
  int homeflag[G];
  int awayflag[G];
  //int nflag[G];
  int<lower=1, upper=N> LW[G,2];  // loser and winner of each game
}

parameters {
  ordered[2] performance[G];
  vector[N] mu;
  real<lower=0> s_mu;
  vector<lower=0>[N] s_pf;
  real H; //ホーム効果
  real A; //アウェー効果
  //real Ne; //ニュートラル
}

model {
  for (g in 1:G)
    performance[g,1] ~ normal(mu[LW[g,1]], s_pf[LW[g,1]]);
  for (g in 1:G)
    performance[g,2] ~ normal(mu[LW[g,2]]+H*homeflag[g]-H*awayflag[g], s_pf[LW[g,2]]);

  mu ~ normal(0, s_mu);
  s_pf ~ gamma(10, 10);
}"""

In [11]:
stan_data = {'N': 366, 'G': len(LW),'LW': LW,"homeflag":season["H"],"awayflag":season["A"],"nflag":season["N"]}
sm = pystan.StanModel(model_code=model)
%time fit = sm.sampling(data=stan_data, iter=1000, chains=3)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_4037e5a291a5ccb5a3226ff3f2d4766c NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


CPU times: user 3.05 s, sys: 11.7 s, total: 14.7 s
Wall time: 34min 16s


In [12]:
fit


For the full summary use 'print(fit)'

Inference for Stan model: anon_model_4037e5a291a5ccb5a3226ff3f2d4766c.
3 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=1500.

                    mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
performance[1,1]   -1.93    0.04   1.08  -4.27   -2.6  -1.89  -1.21    0.1    806   1.01
performance[2,1]   -0.57    0.03   1.19  -3.19  -1.28  -0.51    0.2    1.6   1157    1.0
performance[3,1]   -0.08    0.03   1.02  -2.14  -0.69  -0.11   0.55   2.09   1087    1.0
performance[4,1]   -2.46    0.03   1.03  -4.76  -3.08  -2.35  -1.78  -0.62    964    1.0
performance[5,1]   -2.73    0.03   0.98  -4.87  -3.33  -2.67  -2.07  -0.99   1099    1.0
performance[6,1]   -1.06    0.03   1.03  -3.12  -1.74  -1.01  -0.36    0.8   1314    1.0
performance[7,1]    0.39    0.03   0.96  -1.77  -0.17   0.44   1.03   2.09   1185    1.0
performance[8,1]   -1.02    0.03   1.16  -3.48   -1.7  -0.99 

In [14]:
result = fit.extract()

In [15]:
#ホーム効果
result["H"].mean()

0.41769803078813084

In [16]:
sresult=pd.DataFrame()
sresult["teamid"]=[i+1+3100 for i in range(366)]
sresult["mu"]=np.median(result["mu"],axis=0)
sresult["s_pf"]=np.median(result["s_pf"],axis=0)

In [18]:
sresult.sort_values(by="mu")

,teamid,mu,s_pf
83,3184,-3.427303,0.873460
51,3152,-3.089484,0.874718
153,3254,-2.996693,1.145893
340,3441,-2.878147,0.883892
96,3197,-2.834993,1.040807
7,3108,-2.762938,0.945169
14,3115,-2.738911,0.853811
154,3255,-2.715200,0.915774
269,3370,-2.687803,0.941626
265,3366,-2.636868,1.024879


In [19]:
Wsample=pd.read_csv("../input/WSampleSubmissionStage2.csv")

In [20]:
Wsample["leftteam"]=Wsample["ID"].apply(lambda x:int(x.split("_")[1]))
Wsample["rightteam"]=Wsample["ID"].apply(lambda x:int(x.split("_")[2]))

In [28]:
from scipy.stats import norm
from numpy.random import *
import time
start = time.time()
winrate=[]
for it,row in Wsample.iterrows():
    #print(1,time.time() - start)
    if it%100==0:
        print(it)
    #試行回数
    num=10000
    l_mu=sresult[sresult["teamid"]==row["leftteam"]]["mu"]
    l_s_pf=sresult[sresult["teamid"]==row["leftteam"]]["s_pf"]
    r_mu=sresult[sresult["teamid"]==row["rightteam"]]["mu"]
    r_s_pf=sresult[sresult["teamid"]==row["rightteam"]]["s_pf"]
    #print(2,time.time() - start)
    #空箱
    arr=np.zeros((num,2))
    for n in range(num):
        arr[n,0]=normal(l_mu,l_s_pf)
        arr[n,1]=normal(r_mu,r_s_pf)
    winrate.append(1-(arr.argmax(axis=1).sum()/num))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [29]:
Wsample["Pred"]=np.clip(winrate, 0.025, 0.975)

In [30]:
Wsample[["ID","Pred"]].to_csv("../submit/stanmodelv2_2.csv",index=False)